<a href="https://colab.research.google.com/github/yeonu0649/disc2025/blob/main/%EB%82%A0%EC%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Flask-Caching waitress

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
import requests
import xml.etree.ElementTree as ET
from flask import Flask, jsonify
from flask_caching import Cache

app = Flask(__name__)

# 캐시 설정
app.config['CACHE_TYPE'] = 'SimpleCache'
app.config['CACHE_DEFAULT_TIMEOUT'] = 600 #캐시10분
cache = Cache(app)


service_key = "OEBU5anyrQkL0zi0N1vyjCpBIvoWBYDMB+orxAz7FsyOzDVxU0Bp1YgpSeVnkdfvcbUv2NbRV+O/AEY2mAvD8g=="

# 조회할 공항 코드 목록
AIRPORT_IDS = ["RKSS", "RKSI", "RKNY", "RKTU", "RKPC", "RKPK", "RKJJ", "RKJB"]

def get_weather_data(airport_id):
    """
    특정 공항의 날씨 데이터를 API로 가져오는 함수
    """
    url = f"http://apis.data.go.kr/1360000/MdeMdlService/getMdeMdl?serviceKey={service_key}&airportId={airport_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        root = ET.fromstring(response.content)

        item = root.find('.//item')
        if item:
            temp = item.find('temp').text if item.find('temp') is not None else "N/A"
            wind_dir = item.find('windDir').text if item.find('windDir') is not None else "N/A"
            wind_spd = item.find('windSpd').text if item.find('windSpd') is not None else "N/A"
            return {
                "id": airport_id,
                "temp": temp,
                "wind_dir": wind_dir,
                "wind_spd": wind_spd
            }
        return None
    except Exception as e:
        print(f"Error fetching data for {airport_id}: {e}")
        return None

@app.route('/api/airport-weather')
@cache.cached(timeout=300)
def get_all_airport_weather():
    """
    모든 공항의 날씨 데이터를 JSON 형식으로 반환하는 API 엔드포인트
    """
    all_weather_data = []
    for airport_id in AIRPORT_IDS:
        data = get_weather_data(airport_id)
        if data:
            all_weather_data.append(data)

    return jsonify(all_weather_data)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

if __name__ == '__main__':
    app.run(debug=True)

https://5000-m-s-2o9upeiwqekn0-c.us-central1-0.prod.colab.dev
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
// 날씨 상태에 따라 이모티콘을 반환하는 함수
function getWeatherEmoji(skyDesc) {
    if (skyDesc.includes("맑음")) {
        return "☀️";
    } else if (skyDesc.includes("구름 많음") || skyDesc.includes("흐림")) {
        return "☁️";
    } else if (skyDesc.includes("비")) {
        return "☔";
    } else if (skyDesc.includes("눈")) {
        return "❄️";
    } else {
        return "❓"; // 알 수 없는 날씨
    }
}

async function getAirportWeather() {
    const url = "/api/airport-weather";
    const airportNames = {
        "RKSS": "김포",
        "RKSI": "인천",
        "RKNY": "양양",
        "RKTU": "청주",
        "RKPC": "김해",
        "RKPK": "부산",
        "RKJJ": "제주",
        "RKJB": "무안"
    };

    try {
        const response = await fetch(url);
        const airports = await response.json();

        const weatherDiv = document.getElementById("weatherResult");
        weatherDiv.innerHTML = '<h2>✈️ 국내 주요 공항 기상 정보</h2>';

        if (airports.length === 0) {
            weatherDiv.innerHTML += '<p>❌ 데이터를 가져오는 데 실패했습니다.</p>';
            return;
        }

        airports.forEach(airport => {
            const airportName = airportNames[airport.id] || airport.id;
            const emoji = getWeatherEmoji(airport.sky_desc);

            const airportInfo = document.createElement('div');
            airportInfo.className = 'airport-info';
            airportInfo.innerHTML = `
                <h3>${emoji} ${airportName} 공항</h3>
                <p><strong>날씨:</strong> ${airport.sky_desc}</p>
                <p><strong>온도:</strong> ${airport.temp}°C</p>
                <p><strong>풍향:</strong> ${airport.wind_dir}</p>
                <p><strong>풍속:</strong> ${airport.wind_spd} m/s</p>
            `;
            weatherDiv.appendChild(airportInfo);
        });

    } catch (error) {
        console.error("공항 기상 정보를 가져오는 중 오류 발생:", error);
        document.getElementById("weatherResult").innerHTML = `<p>❌ 공항 기상 정보를 불러오지 못했습니다.</p>`;
    }
}

getAirportWeather();

In [ ]:
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8" />
    <title>공항 날씨 정보</title>
    <link rel="stylesheet" href="style.css" />
</head>
<body>
    <h1>✈️ 국내 주요 공항 기상 정보</h1>
    <div id="weatherResult"></div>
    <script src="script.js"></script>
</body>
</html>

In [ ]:
body {
    font-family: 'Noto Sans KR', sans-serif;
    text-align: center;
    background-color: #f0f8ff;
    padding: 40px;
}

h1 {
    color: #007bff;
    margin-bottom: 40px;
}

#weatherResult {
    margin-top: 30px;
    display: flex;
    flex-wrap: wrap;
    justify-content: center;
    gap: 20px;
}

.airport-info {
    border: 1px solid #ddd;
    border-radius: 8px;
    padding: 20px;
    width: 200px;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    background-color: #fff;
    text-align: left;
}

.airport-info h3 {
    margin-top: 0;
    color: #333;
    font-size: 1.2rem;
    display: flex;
    align-items: center;
    gap: 10px;
}

.airport-info p {
    margin: 5px 0;
    font-size: 16px;
}